# Pump it Up: Data Mining the Water Table

By: [Ville Heilala](https://heila.la)

Datasource: http://taarifa.org/, http://maji.go.tz/, https://www.drivendata.org

Goal is to predict the operating condition of a waterpoint for each record in the dataset.

## Preprocess the data

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 12:22:00) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Pandas version 0.19.2
Matplotlib version 2.0.0


In [7]:
######################
# Load data
##################################################################

# Read data
train_values = pd.read_csv("/train_values.csv")
train_labels = pd.read_csv("/train_labels.csv")
test_values = pd.read_csv("/test_values.csv")

# Add binary variables for train set and test values
train_values["train"] = True
test_values["test"] = True

# Merge train values and test values
data = pd.concat([train_values, test_values], ignore_index = True)

In [8]:
from math import log
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
import geocoder

# latitude, longitude: round
cols = ["latitude",
        "longitude"]

for col in cols:
    data[col] = data[col].map(lambda x: round(x, 7))
    data[col] = data[col].replace(to_replace=-0.0, value=0)

# Get missing latitude and longitude values
for lga in data[(data.latitude == 0) | (data.longitude == 0)]["lga"].unique():
    g = geocoder.google(lga)
    data.loc[data.lga == lga, "latitude"] = g.latlng[0]
    data.loc[data.lga == lga, "longitude"] = g.latlng[1]

# Get missing gps_height values
heights = pd.read_csv("heights.csv")
heights = pd.Series(heights.gps_height.values, index=heights.id).to_dict()
for key in heights.keys():
    data.loc[data.id == key, "gps_height"] = heights[key]
    
# Shorten and combine
cols = [["installer", 3, 500],
        ["funder", 3, 50],
        ["subvillage", 4, 300],
        ["scheme_name", 5, 300]]
combined = "oth"

for col, chars, treshold in cols:
    # Missing values to combined group
    data.loc[data[col].isnull() | data[col].isin(["0", "-", "_"]), col] = col + "_" + combined
    # Lowercase and shorten
    data[col] = data[col].map(lambda x: x[:chars].lower())
    # Combine lower than treshold
    data.loc[data[col].isin(data[col].value_counts()[data[col].value_counts() < treshold].index), col] = col + "_" + combined

drop = ["date_recorded",
        "recorded_by",
        "ward",
        "wpt_name"]
data.drop(drop, axis=1, inplace=True)

data["amount_tsh"] = data["amount_tsh"].apply(lambda x: log(round(x)) if round(x) > 0 else 0)
data["population"] = data["population"].apply(lambda x: log(x) if x > 0 else 0)

# latitude, longitude: round
cols = ["latitude",
        "longitude"]

for col in cols:
    data[col] = data[col].map(lambda x: round(x, 6))
    data[col] = data[col].replace(to_replace=-0.0, value=0)

# DBSCAN
db = DBSCAN(eps=0.2, min_samples=200)
data["loc_clust"] = db.fit_predict(data[["latitude", "longitude"]])
    
data["district_code"] = data["district_code"].map(lambda x: str(x))
data["construction_year"] = data["construction_year"].map(lambda x: str(x))
data["region_code"] = data["region_code"].map(lambda x: str(x))
data["loc_clust"] = data["loc_clust"].map(lambda x: str(x))
data["num_private"] = data["num_private"].map(lambda x: str(x))

normalize = ['gps_height',
             "amount_tsh",
             "population",
             "latitude",
             "longitude"]
scaler = MinMaxScaler(feature_range=(0, 1))
for n in normalize:
    rescaledX = scaler.fit_transform(data[[n]])
    data[n] = rescaledX
    
dummify = ['basin',
          'construction_year',
          'district_code',
          'extraction_type',
          'extraction_type_class',
          'extraction_type_group',
          'funder',
          'installer',
          'lga',
          'management',
          'management_group',
          'num_private',
          'payment',
          'payment_type',
          'permit',
          'public_meeting',
          'quality_group',
          'quantity',
          'quantity_group',
          'region',
          'region_code',
          'scheme_management',
          'scheme_name',
          'source',
          'source_class',
          'source_type',
          'subvillage',
          'water_quality',
          'waterpoint_type',
          'waterpoint_type_group',
          'loc_clust']
    
final = pd.get_dummies(data[dummify], dummy_na = True)
conc = [c for c in list(data) if c not in dummify]
final = pd.concat([data[conc], final], axis = 1)

# Subset train values
train_values = final[final["train"] == True]
# Merge train labels
train_values = pd.merge(train_values, train_labels, on="id")

# Subset test values
test_values = final[final["test"] == True]

In [14]:
train_values.drop("train", axis=1, inplace=True)
train_values.shape
test_values.drop("test", axis=1, inplace=True)
test_values.shape

(59400, 750)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


(14850, 749)

In [17]:
train_values.to_csv("/train_values_processed.csv", index = False)
test_values.to_csv("/test_values_processed.csv", index = False)